# T9

In [28]:
import numpy as np

import matplotlib.pyplot as plt
from matplotlib import ticker
import seaborn as sns

from math import exp, pi, sqrt

from scipy.integrate import quad
from scipy.special import gamma
from scipy.optimize import minimize
import scipy.stats as sps

from tqdm import tqdm

## a) Проверка гипотезы о согласии данных с законом равномерного распредления с помощью критерия $\chi^2$ и с помощью критерия Колмогорова

### Считаем интеграл для p-value

In [2]:
def density(x, n):
    return 0.5 ** (n / 2) / gamma(n / 2) * x ** (n / 2 - 1) * exp(-0.5 * x)

In [3]:
I = quad(density, 16.4, float('inf'), args=(9))
I

(0.058984030544419676, 2.513395417257353e-10)

### Критерий Колмогорова

In [4]:
# наша выборка
sample = [5, 8, 6, 12, 14, 18, 11, 6, 13, 7]

In [5]:
# получаем значения эмпирической функции распределения
n = 100

In [6]:
def calculate_edf(sample, n):
    frequencies = np.array(sample) / 100
    edf = np.array([np.sum(frequencies[:i + 1]) for i in range(len(sample))])
    return edf

In [7]:
edf = calculate_edf(sample, n)
edf

array([0.05, 0.13, 0.19, 0.31, 0.45, 0.63, 0.74, 0.8 , 0.93, 1.  ])

In [8]:
def uniform_density(a, b, x):
    """Равномерное распределение"""
    return (x - a) / (b - a)

In [9]:
def kolmogorov_density(x):
    """Колмогоровское распределение"""
    sum_result = 0
    for i in range(1, 10000):
        sum_result += (-1) ** i * exp(-2 * i ** 2 * x ** 2)
    return 1 + 2 * sum_result

In [10]:
def calculate_delta(sample, n, edf):
    previous_height = 0
    delta = -1
    for i in range(len(sample)):
        density_value = uniform_density(0, 9, i)
        delta = max(abs(density_value - edf[i]), abs(density_value - previous_height), delta)
        previous_height = edf[i]
    delta *= sqrt(n)
    return delta

In [12]:
delta_kolm = calculate_delta(sample, n, edf) # делта для колмогоровского распределения
print(f"Дельта критерия Колмогорова: {delta_kolm}")

Дельта критерия Колмогорова: 1.4333333333333331


In [14]:
p_value_kolm = 1 - kolmogorov_density(delta_kolm)
print(f"p-value критерия Колмогорова: {p_value_kolm}")

p-value критерия Колмогорова: 0.032851885438597406


## b) Проверка гипотезы о согласии данных с законом нормального распределения с помощью критерий $\chi^2$ и с помощью критерия Колмогорова

In [15]:
def normal_density(x, tetha_1, tetha_2):
    return 1 / (sqrt(2 * pi) *  tetha_2) * exp(-((x - tetha_1) ** 2) / ((tetha_2 ** 2) * 2))


def normal_function(x, tetha_1, tetha_2):
    return quad(normal_density, x[0], x[1], args=(tetha_1, tetha_2))[0]

In [16]:
def likelihood_function(parameters, sample):
    mean, sigma = parameters
    intervals = [[-float('inf'), 1]] + [[i + 1, i + 2] for i in range(8)] + [[9, float('inf')]]

    func = 1
    for i in range(len(intervals)):
        func *= normal_function(intervals[i], mean, sigma) ** sample[i]
    
    return -func  # минус для того, что бы найти максимум с помощью функции, которая ищет минимум

In [17]:
def find_parameters(sample):
    start_point = [3, 2]
    result = minimize(likelihood_function, start_point, args=(sample), method='Nelder-Mead')
    mean, sigma = result.x
    return mean, sigma

In [18]:
mean, sigma = find_parameters(sample)
print(f'Оценки параметров ОМПГ:\nСреднее = {mean}, Отклонение = {sigma}')

Оценки параметров ОМПГ:
Среднее = 5.289737726548878, Отклонение = 2.6795139919364033


In [19]:
intervals = [[-float('inf'), 1]] + [[i + 1, i + 2] for i in range(8)] + [[9, float('inf')]]
delta = 0

for i in range(len(sample)):
    np_ = n * normal_function(intervals[i], mean, sigma)
    delta += (sample[i] - np_) ** 2 / np_
print(f'Дельта для критерия χ²: {delta}')

Дельта для критерия χ²: 9.802444291486268


In [20]:
p_value_ompg = quad(density, delta, float('inf'), args=(7))[0]
print(f"p-value критерия χ²: {p_value_ompg}")

p-value критерия χ²: 0.20004793606221863


### Проверка по критерию Колмогорову

Для критерия Колмогорова будем использовать ОМПГ оценку параметров

In [21]:
N = 50000
borders =  [-float('inf'), 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, float('inf')]

In [22]:
def calculate_sample(bootstrap_sample):
    sample = []
    for i in range(len(borders) - 1):
        a = bootstrap_sample[bootstrap_sample < borders[i + 1]]
        a = a[a > borders[i]]
        sample.append(len(a))
    return sample

In [41]:
def calculate_delta_bootstrap(mean, sigma, edf):
    previous_height = 0
    delta = -1
    for i in range(len(intervals)):
        density_value = sps.norm.cdf(i, loc=mean, scale=sigma)
        delta = max(abs(density_value - edf[i]), abs(density_value - previous_height), delta)
        previous_height = edf[i]
    delta *= sqrt(n)
    return delta

In [42]:
bootstrap_deltas = []

k = 0

progressbar = tqdm(range(N), desc='Kolmogorov Bootstrap')

for i in progressbar:
    bootstrap_sample = sps.norm(loc=mean, scale=sigma).rvs(100)
    calculated_sample = calculate_sample(bootstrap_sample)
    mean_bootstrap, sigma_bootstrap = find_parameters(calculated_sample)
    edf = calculate_edf(calculated_sample, 100)
    delta_tmp = calculate_delta_bootstrap(mean_bootstrap, sigma_bootstrap, edf)
    # print(delta_tmp)
    if delta_tmp > delta_kolm:
        k += 1
        
    progressbar.set_description(f"Kolmogorov Bootstrap p-value: {k / N}")

Kolmogorov Bootstrap p-value: 0.2391: 100%|██████████| 50000/50000 [44:24<00:00, 18.77it/s] 


In [46]:
print(f"p-value по критерию Колмогорова: {k / N}")

p-value по критерию Колмогорова: 0.2391
